# GNN notebook

In [5]:
from typing import List
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import jraph
import optax
import data_utils
import os
import pathlib
import h5py
from torch.utils.data import Dataset, DataLoader, default_collate


os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [9]:
train_dir = "/global/cfs/projectdirs/m3246/mingfong/transfer-learning/delphes_train_set/"
train_path = pathlib.Path(train_dir)
# train_file_names = os.listdir(train_dir)
train_filepaths = [p for p in train_path.glob("*.h5")]
train_filepaths

[PosixPath('/global/cfs/projectdirs/m3246/mingfong/transfer-learning/delphes_train_set/train_6.h5'),
 PosixPath('/global/cfs/projectdirs/m3246/mingfong/transfer-learning/delphes_train_set/train_2.h5'),
 PosixPath('/global/cfs/projectdirs/m3246/mingfong/transfer-learning/delphes_train_set/train_8.h5'),
 PosixPath('/global/cfs/projectdirs/m3246/mingfong/transfer-learning/delphes_train_set/train_12.h5'),
 PosixPath('/global/cfs/projectdirs/m3246/mingfong/transfer-learning/delphes_train_set/train_4.h5'),
 PosixPath('/global/cfs/projectdirs/m3246/mingfong/transfer-learning/delphes_train_set/train_11.h5'),
 PosixPath('/global/cfs/projectdirs/m3246/mingfong/transfer-learning/delphes_train_set/train_5.h5'),
 PosixPath('/global/cfs/projectdirs/m3246/mingfong/transfer-learning/delphes_train_set/train_7.h5'),
 PosixPath('/global/cfs/projectdirs/m3246/mingfong/transfer-learning/delphes_train_set/train_14.h5'),
 PosixPath('/global/cfs/projectdirs/m3246/mingfong/transfer-learning/delphes_train_set/t

In [ ]:
class H5GraphDataset(Dataset):
  """
  Turns h5 files into GraphsTuple objects
  """
  def __init__(self, filepaths: List[str], max_rows:int = None):
    self.filepaths = filepaths

In [7]:
# create GraphsTuple
MAX_CONSTITUENTS = 200      # nodes per graph

with h5py.File(str(train_filepaths[0]), "r") as f:
  col_names = list(f.keys())
  file_len = len(f["labels"])
  for col_name in col_names:
    print(col_name, f[col_name].shape)
  
  node_features, edge_features, senders, receivers, globals, targets = None, None, None, None, None, None
  

fjet_clus_E (5000000, 200)
fjet_clus_eta (5000000, 200)
fjet_clus_phi (5000000, 200)
fjet_clus_pt (5000000, 200)
fjet_eta (5000000,)
fjet_m (5000000,)
fjet_phi (5000000,)
fjet_pt (5000000,)
labels (5000000,)
training_weights (5000000,)


In [8]:
# classification example
# https://github.com/google-deepmind/jraph/blob/51f5990104f7374492f8f3ea1cbc47feb411c69c/jraph/ogb_examples/train.py#L89